### Prepare training dataset for ensemble learning

In [ ]:
import os, sys
import h5py
import pandas as pd
import numpy as np
import time

sys.path.append('../../ELEP/src/')
from mbf_utils import make_LogFq, make_LinFq, rec_filter_coeff, create_obspy_trace
from mbf import MB_filter as MBF

# If you use multiprocessors, please install mpi4py and run it with mpirun locally
# from mpi4py import MPI 

In [ ]:
# prepare raw data 
flnm = ['Instance_snr_18_20_testing.csv']
dirpath_csv  = # define the path to the metadata in .csv file 
dirpath_data = # define the path to the waveforms in .h5 file

dirpath_csv_save = # define the path to save the metadata in .csv file
dirpath_data_save = # define the path to save the waveforms in .h5 file

In [ ]:
for ifl in range (rank,len(flnm_list),size):
    csv_fl = flnm_list[ifl]
    print(f'rank={rank}, working on file={csv_fl}')

    # write into a new h5
    dirpath_csv = '/Volumes/GD-cyuan/DataYuan/INSTANCE/dataset/processed/Ensemble/noisy_INSTANCE/'
    dirpath_data = '/Volumes/GD-cyuan/DataYuan/INSTANCE/dataset/processed/Ensemble/INSTANCE_data/'

    # set up parameters for multi-band frequency filtering
    npts, dt, fs, nc, mtraces = 6000, 0.01, 100., 3, 20000
    components = ['E','N','Z']
    MBF_filter_paras = {'f_min':1, 'f_max':45, 'nfbs':10, 'frequencies':[], 'dt':0.01, 'CN_HP':[], 'CN_LP':[], 'npoles':2} 
    MBF_filter_paras['frequencies'] = make_LogFq(MBF_filter_paras['f_min'], MBF_filter_paras['f_max'], MBF_filter_paras['dt'], MBF_filter_paras['nfbs'])
    MBF_filter_paras['CN_HP'], MBF_filter_paras['CN_LP'] = rec_filter_coeff(MBF_filter_paras['frequencies'], MBF_filter_paras['dt'])
    freq_list = MBF_filter_paras['frequencies']
    MBF_filter_paras['detection_threshold'], MBF_filter_paras['P_threshold'], MBF_filter_paras['S_threshold'] = 0.3, 0.1, 0.1
    MBF_filter_paras['estimate_uncertainty'] = None
    nfbs = len(freq_list-1)
    print(freq_list)

    snr = int(csv_fl.split('_')[-2])
    dirpath_wav = '/Volumes/GD-cyuan/DataYuan/INSTANCE/dataset/processed/metadata_STEAD_events_STEAD_noise_snr0_20_full.hdf5'

    # change name with csv file name
    pick_prob_fnm_mbf = dirpath_data+csv_fl[:-4]+'_mbf_wav.hdf5'

    h5_wdata2 = h5py.File(pick_prob_fnm_mbf, 'a')
    h5_wdata2.create_group('data') 

    # open h5 reader
    h5_rdata = h5py.File(dirpath_wav, 'r')

    # read csv for trace names
    csv_reader = pd.read_csv(dirpath_csv+csv_fl)
    trace_name_list = csv_reader.trace_name.values

    # start prediction
    for itr, trnm in enumerate(trace_name_list[:mtraces]): 
        if itr%2000 == 0:
            print(f'working on itr={itr}')
    
        # get data
        dataset = h5_rdata.get('data/'+str(trnm)) 
        data = np.array(dataset)

        # MBF filtering
        mbf_clear_data = np.zeros([nfbs, npts, nc])
        for ic in range(nc):
            mbf_clear_data[:, :, ic] = MBF(data[ic], MBF_filter_paras)
      
        # save data
        h5_wdata2 = h5py.File(pick_prob_fnm_mbf, 'r')
        h5w2 = h5_wdata2.create_dataset('data/'+trnm, mbf_clear_data.shape, data = mbf_clear_data, dtype= np.float32) 
        h5w2.flush() 

    # h5_wdata.close()
    h5_wdata2.close()
    h5_rdata.close()
